In [1]:
import pandas as pd
import numpy as np
import sqlite3
import time

In [2]:
hopteaming = pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', nrows = 100)
taxonomy = pd.read_csv('data/nucc_taxonomy_240.csv', nrows = 100)
zip_cbsa = pd.read_csv('data/ZIP_CBSA_122023.csv', dtype={'ZIP': str}, nrows= 100)

In [3]:
npps_all = pd.read_csv('data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv',
                       dtype={'Provider Business Practice Location Address Postal Code': str}, 
                       usecols=['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text', 'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code', 
                               'Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1', 'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2', 'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3', 'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4', 'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5', 'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6', 'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7', 'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8', 'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9', 'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10', 'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11', 'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12', 'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13', 'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14', 'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15',
                               'Healthcare Provider Taxonomy Group_1', 'Healthcare Provider Taxonomy Group_2', 'Healthcare Provider Taxonomy Group_3', 'Healthcare Provider Taxonomy Group_4', 'Healthcare Provider Taxonomy Group_5', 'Healthcare Provider Taxonomy Group_6', 'Healthcare Provider Taxonomy Group_7', 'Healthcare Provider Taxonomy Group_8', 'Healthcare Provider Taxonomy Group_9', 'Healthcare Provider Taxonomy Group_10', 'Healthcare Provider Taxonomy Group_11', 'Healthcare Provider Taxonomy Group_12', 'Healthcare Provider Taxonomy Group_13', 'Healthcare Provider Taxonomy Group_14', 'Healthcare Provider Taxonomy Group_15'],
                       nrows=100)

npps_all.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,...,Healthcare Provider Taxonomy Group_6,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15
0,1679576722,1.0,NaN,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
taxonomy_cols = [col for col in npps_all.columns if 'Taxonomy' in col]
taxonomy_switch_list = [col for col in npps_all.columns if 'Taxonomy Switch' in col]
taxonomy_code_list = [col for col in npps_all.columns if 'Taxonomy Code' in col]

In [5]:
npps_all[taxonomy_cols]

,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,...,Healthcare Provider Taxonomy Group_6,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15
0,207X00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,207RC0000X,Y,207RC0000X,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,251G00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,174400000X,N,207RH0003X,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,207V00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,261QF0400X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,261QA1903X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,363AM0700X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
npps_all['Healthcare Provider Taxonomy Code'] = np.nan
for (code, switch) in zip(taxonomy_code_list, taxonomy_switch_list):
    npps_all['Healthcare Provider Taxonomy Code'] = np.where(npps_all[switch] == 'Y', npps_all[code], npps_all['Healthcare Provider Taxonomy Code'])
#npps_all['taxonomy_codes_clean'] = np.where(npps_all['Healthcare Provider Primary Taxonomy Switch_1'] == 'Y', npps_all['Healthcare Provider Taxonomy Code_1'], np.nan)

In [7]:
npps_all['Healthcare Provider Taxonomy Code']

0     207X00000X
1     207RC0000X
2     251G00000X
3            NaN
4     207RH0003X
         ...    
95    207V00000X
96    261QF0400X
97    261QA1903X
98    363AM0700X
99    207Q00000X
Name: Healthcare Provider Taxonomy Code, Length: 100, dtype: object

In [8]:
npi = npps_all[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text', 'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code']]

In [9]:
npi

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Healthcare Provider Taxonomy Code
0,1679576722,1.0,NaN,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944,207X00000X
1,1588667638,1.0,NaN,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552,251G00000X
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243,207RH0003X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1649273673,1.0,NaN,TODD,MICHAEL,NaN,NaN,MD,3311 BEAM RD,STE 300A,CHARLOTTE,NC,282179422,207V00000X
96,1083617013,2.0,AMOSKEAG HEALTH,NaN,NaN,NaN,NaN,NaN,145 HOLLIS ST,NaN,MANCHESTER,NH,031011325,261QF0400X
97,1992708929,2.0,NOVAMED MANAGEMENT SERVICES LLC,NaN,NaN,NaN,NaN,NaN,3200 DOWNWOOD CIR NW,STE 200,ATLANTA,GA,303271610,261QA1903X
98,1801899836,1.0,NaN,SARAH,L,MRS.,NaN,PA-C,2801 K ST,SUITE 410,SACRAMENTO,CA,958165120,363AM0700X


In [10]:
zip_cbsa.head()

,ZIP,CBSA,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,00501,35620,HOLTSVILLE,NY,0.0,1.0,0.0,1.0
1,00601,38660,ADJUNTAS,PR,1.0,1.0,1.0,1.0
2,00602,10380,AGUADA,PR,1.0,1.0,1.0,1.0
3,00603,10380,AGUADILLA,PR,1.0,1.0,1.0,1.0
4,00604,10380,AGUADILLA,PR,1.0,1.0,1.0,1.0


In [11]:
taxonomy.head()

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual


In [12]:
hopteaming = hopteaming[(hopteaming.transaction_count >=50) & (hopteaming.average_day_wait <50)]
hopteaming.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
7,1508085911,1730166125,58,67,23.925,43.923
10,1508167040,1730166125,51,51,28.196,52.876
15,1508863549,1730166125,340,391,18.302,42.422
18,1508867870,1730166125,50,79,12.658,26.402
25,1508011040,1730166224,132,145,8.579,28.053


In [13]:
start = time.time()
db = sqlite3.connect('data/hop_teaming.sqlite')
for chunk in pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', 
                         chunksize = 10000):
    chunk = chunk[(chunk.transaction_count >=50) & (chunk.average_day_wait <50)]
    chunk.columns = [x.lower().replace(' ','_') for x in chunk.columns]
    chunk.to_sql('hopteaming', db, if_exists = 'append', index = False)
db.close()

end = time.time()    
print(f'This took {end-start} seconds to run')

This took 665.7662134170532 seconds to run


In [14]:
start = time.time()
db = sqlite3.connect('data/hop_teaming.sqlite')
for chunk in pd.read_csv('data/NPPES_Data_Dissemination_February_2024/npidata_pfile_20050523-20240211.csv',
                         dtype={'Provider Business Practice Location Address Postal Code': str}, 
                         low_memory = False,
                         chunksize = 10000):
    chunk['Healthcare Provider Taxonomy Code'] = np.nan
    
    #Taxonomy Code Consolidation Loop Starts Here
    for (code, switch) in zip(taxonomy_code_list, taxonomy_switch_list):
        chunk['Healthcare Provider Taxonomy Code'] = np.where(chunk[switch] == 'Y', chunk[code], chunk['Healthcare Provider Taxonomy Code'])
    #Taxonomy Code Consolidation Loop Ends Here
    
    #Make a zipcode column that is a numeric 5 digit zip code
    chunk['zip'] = pd.to_numeric(chunk['Provider Business Practice Location Address Postal Code'].str[:5], errors='coerce')
    
    #Select Columns for final table and read in as "chunk2" instead of "chunk"
    chunk2 = chunk[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text', 'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code', 'Healthcare Provider Taxonomy Code', 'zip']]
    
    
    chunk2.columns = [x.lower().replace(' ','_') for x in chunk2.columns]
    chunk2.to_sql('npi', db, if_exists = 'append', index = False)
db.close()

end = time.time()    
print(f'This took {end-start} seconds to run')

This took 452.3672139644623 seconds to run


In [15]:
start = time.time()
db = sqlite3.connect('data/hop_teaming.sqlite')
for chunk in pd.read_csv('data/nucc_taxonomy_240.csv', 
                          chunksize = 10000):
    chunk.columns = [x.lower().replace(' ','_') for x in chunk.columns]
    chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)
db.close()

end = time.time()    
print(f'This took {end-start} seconds to run')

This took 0.04801011085510254 seconds to run


In [16]:
start = time.time()
db = sqlite3.connect('data/hop_teaming.sqlite')
for chunk in pd.read_csv('data/ZIP_CBSA_122023.csv',
                         dtype={'ZIP': str},
                         chunksize = 10000):
    chunk.columns = [x.lower().replace(' ','_') for x in chunk.columns]
    chunk.to_sql('zip_cbsa', db, if_exists = 'append', index = False)
db.close()

end = time.time()    
print(f'This took {end-start} seconds to run')

This took 0.3250436782836914 seconds to run


In [17]:
#Create indexes on columns that are used for joins and filters
start = time.time()

db = sqlite3.connect('data/hop_teaming.sqlite')

db.execute('CREATE INDEX from_npi ON hopteaming(from_npi)')
db.execute('CREATE INDEX to_npi ON hopteaming(to_npi)')

db.execute('CREATE INDEX code ON taxonomy(code)')

db.execute('CREATE INDEX healthcare_provider_taxonomy_code ON npi(healthcare_provider_taxonomy_code)')
db.execute('CREATE INDEX zip ON npi(zip)')
db.execute('CREATE INDEX index_npi ON npi(npi)')

db.execute('CREATE INDEX cbsa ON zip_cbsa(cbsa)')

db.close()

end = time.time()    
print(f'This took {end-start} seconds to run')

This took 159.34267377853394 seconds to run


In [18]:
#ONLY USE FOR DROPPING TABLES
#CHECK TO ENSURE YOU ARE DROPPING THE RIGHT TABLE
# "ctrl" + "/" to uncomment or re-comment the block of code below


# db.close()
# connection = sqlite3.connect('data/hop_teaming.sqlite') 

# # drop table 
# connection.execute("DROP TABLE npi") 

# print("data dropped successfully") 

# # close the connection 
# connection.close() 

In [19]:
query = "SELECT * FROM zip_cbsa LIMIT 10"

with sqlite3.connect('data/hop_teaming.sqlite') as db: 
    zip_cbsa_sqlite = pd.read_sql(query, db)

zip_cbsa_sqlite.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,00501,35620,HOLTSVILLE,NY,0.0,1.0,0.0,1.0
1,00601,38660,ADJUNTAS,PR,1.0,1.0,1.0,1.0
2,00602,10380,AGUADA,PR,1.0,1.0,1.0,1.0
3,00603,10380,AGUADILLA,PR,1.0,1.0,1.0,1.0
4,00604,10380,AGUADILLA,PR,1.0,1.0,1.0,1.0


In [20]:
query = "SELECT * FROM npi LIMIT 10"

with sqlite3.connect('data/hop_teaming.sqlite') as db: 
    npi_sqlite = pd.read_sql(query, db)
    
npi_sqlite.head()

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,healthcare_provider_taxonomy_code,zip
0,1679576722,1.0,None,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,688472944,207X00000X,68847.0
1,1588667638,1.0,None,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736,207RC0000X,32204.0
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,283044552,251G00000X,28304.0
3,1306849450,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN
4,1215930367,1.0,None,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,770901243,207RH0003X,77090.0


In [21]:
query = "SELECT * FROM taxonomy LIMIT 10"

with sqlite3.connect('data/hop_teaming.sqlite') as db: 
    taxonomy_sqlite = pd.read_sql(query, db)
    
taxonomy_sqlite.head()

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual


In [22]:
query = "SELECT * FROM hopteaming LIMIT 10"

with sqlite3.connect('data/hop_teaming.sqlite') as db: 
    hopteaming_sqlite = pd.read_sql(query, db)
    
hopteaming_sqlite.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508085911,1730166125,58,67,23.925,43.923
1,1508167040,1730166125,51,51,28.196,52.876
2,1508863549,1730166125,340,391,18.302,42.422
3,1508867870,1730166125,50,79,12.658,26.402
4,1508011040,1730166224,132,145,8.579,28.053
